In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.Builder().appName("automatic backfill").getOrCreate()

25/04/20 12:03:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from pyspark.sql.types import StructType, StructField, TimestampType, FloatType, StringType

schema = StructType([
    StructField("datetime", TimestampType(), True),
    StructField("temperature", FloatType(), True),
    StructField("humidity", FloatType(), True),
    StructField("file_path", StringType(), True)
])

In [3]:
from pyspark.sql.functions import input_file_name

# Read all raw incoming CSV files from the directory
raw_df = spark.read.option("header", "true").schema(schema).csv("./automatic_backfill/data/raw/*.csv")

# Add respective file path to each row
raw_df = raw_df.withColumn("file_path", input_file_name())

# Show the raw data
raw_df.show(5, truncate=False)

+-------------------+-----------+--------+------------------------------------------------------------+
|datetime           |temperature|humidity|file_path                                                   |
+-------------------+-----------+--------+------------------------------------------------------------+
|2025-05-01 00:00:00|18.5       |65.2    |file:///workspace/automatic_backfill/data/raw/2025-05-01.csv|
|2025-05-01 01:00:00|17.9       |67.5    |file:///workspace/automatic_backfill/data/raw/2025-05-01.csv|
|2025-05-01 02:00:00|17.2       |69.8    |file:///workspace/automatic_backfill/data/raw/2025-05-01.csv|
|2025-05-01 03:00:00|16.8       |71.0    |file:///workspace/automatic_backfill/data/raw/2025-05-01.csv|
|2025-05-01 04:00:00|16.5       |72.3    |file:///workspace/automatic_backfill/data/raw/2025-05-01.csv|
+-------------------+-----------+--------+------------------------------------------------------------+
only showing top 5 rows



In [4]:
# Read all bronze CSV files
bronze_df = spark.read.csv("./automatic_backfill/data/bronze/", header=True, schema=schema)

# Get distinct file paths
bronze_df = bronze_df.select("file_path").distinct()

bronze_df.show(truncate=False)

25/04/20 12:04:30 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 3) (0.0.0.0 executor 0): FetchFailed(BlockManagerId(1, 0.0.0.0, 38871, None), shuffleId=0, mapIndex=1, mapId=2, reduceId=117, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:437)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1242)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:971)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:86)
	at org.apache.spark.util.CompletionIterator.next(CompletionIterator.scala:29)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.util.CompletionIterator.hasNext(Co

Py4JJavaError: An error occurred while calling o72.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 3 (showString at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:437)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1242)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:971)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:86)
	at org.apache.spark.util.CompletionIterator.next(CompletionIterator.scala:29)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterat

In [10]:
df.write.mode("overwrite").csv("./automatic_backfill/data/bronze/")